In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import LogisticRegression

In [2]:
dataframe = fetch_20newsgroups()
# dataframe

In [3]:
# Dividimos el DataFrame en datos y etiquetas
X = dataframe['data']
y = dataframe['target']

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [5]:
vectorizer = CountVectorizer(stop_words='english', max_features=10000)

classifiers = [
    ('naive_bayes', MultinomialNB()),
    ('logistic_regression', LogisticRegression()),
    ('support_vector_classifier', SVC()),
    ('tree', DecisionTreeClassifier()),
    ('neighbors', KNeighborsClassifier()),
    ('ridge_classifier', RidgeClassifier()),
    ('SGD_classifier', SGDClassifier())
]

In [6]:
pipelines = [
    Pipeline([
        ('vect', vectorizer),
        ('clf', clf[1])
    ])
    for clf
    in classifiers
]

for pl in pipelines:
    pl.fit(X_train, y_train)
    

/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
predictions = [
    pl.predict(X_test)
    for pl
    in pipelines
]

accuracy = {
    cl[0]: accuracy_score(y_test, p)
    for cl, p
    in zip(classifiers, predictions)
}

accuracy

{'naive_bayes': 0.8405797101449275,
 'logistic_regression': 0.8738069989395546,
 'support_vector_classifier': 0.08660303994344291,
 'tree': 0.6595970307529162,
 'neighbors': 0.4796747967479675,
 'ridge_classifier': 0.7242841993637328,
 'SGD_classifier': 0.8296217744786144}

In [8]:
pd.DataFrame(((a[0], a[1]) for a in accuracy.items()), columns=["method", "accuracy"]).sort_values("accuracy", ascending=False)

,method,accuracy
1,logistic_regression,0.873807
0,naive_bayes,0.840580
6,SGD_classifier,0.829622
5,ridge_classifier,0.724284
3,tree,0.659597
4,neighbors,0.479675
2,support_vector_classifier,0.086603


In [9]:
classifiers = [
    ('ridge_classifier', RidgeClassifier()),
    ('SGD_classifier', SGDClassifier())
]

In [10]:
pd.DataFrame(((a[0], a[1]) for a in accuracy.items()), columns=["method", "accuracy"]).sort_values("accuracy", ascending=False)

,method,accuracy
1,logistic_regression,0.873807
0,naive_bayes,0.840580
6,SGD_classifier,0.829622
5,ridge_classifier,0.724284
3,tree,0.659597
4,neighbors,0.479675
2,support_vector_classifier,0.086603


In [11]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

In [12]:
test_params = {
    'loss': ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'penalty': ['l1', 'l2', 'elasticnet'],
}

vectorizer.fit(X_train)
Xvec_train = vectorizer.transform(X_train)

clf = SGDClassifier()

search = HalvingRandomSearchCV(clf, test_params, random_state=42)
search.fit(Xvec_train, y_train)
search.best_params_

/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 27 is smaller than n_iter=42. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


{'penalty': 'l2', 'loss': 'perceptron'}

In [13]:
best_clf = SGDClassifier(**search.best_params_)
best_clf.fit(Xvec_train, y_train)

Xvec_test = vectorizer.transform(X_test)

prediction = best_clf.predict(Xvec_test)

print(f"Accuracy: {accuracy_score(prediction, y_test)}")

Accuracy: 0.8303287380699894


In [14]:
from sklearn.model_selection import GridSearchCV

test_params = {
    'solver': ['newton-cg', 'sag', 'saga', 'lbfgs'],
    'penalty': ['l2', None],
}

vectorizer.fit(X_train)
Xvec_train = vectorizer.transform(X_train)

clf = LogisticRegression()

search = GridSearchCV(clf, test_params)
search.fit(Xvec_train, y_train)
search.best_params_

/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn

{'penalty': 'l2', 'solver': 'lbfgs'}

In [15]:
# Creamos un vectorizador y un clasificador Naive Bayes
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
clf = MultinomialNB()

# Creamos un pipeline que incluya el vectorizador y el clasificador
pipeline = Pipeline([
    ('vect', vectorizer),
    ('clf', clf)
])

# Entrenamos el modelo
pipeline.fit(X_train, y_train)

# Comprobar los resultados con los datos de prueba
predictions = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

Accuracy: 0.8405797101449275
              precision    recall  f1-score   support

           0       0.89      0.94      0.92       117
           1       0.62      0.83      0.71       138
           2       1.00      0.03      0.05       142
           3       0.53      0.81      0.64       145
           4       0.77      0.85      0.81       157
           5       0.78      0.85      0.82       136
           6       0.72      0.82      0.77       137
           7       0.88      0.90      0.89       164
           8       0.89      0.94      0.91       155
           9       0.91      0.96      0.93       141
          10       0.98      0.93      0.95       136
          11       1.00      0.88      0.94       156
          12       0.86      0.83      0.84       144
          13       0.93      0.93      0.93       145
          14       0.95      0.91      0.93       152
          15       0.93      0.91      0.92       150
          16       0.93      0.94      0.93       15

In [16]:

# Creamos un vectorizador y un clasificador Naive Bayes
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
clf = LogisticRegression()

# Creamos un pipeline que incluya el vectorizador y el clasificador
pipeline = Pipeline([
    ('vect', vectorizer),
    ('clf', clf)
])

# Entrenamos el modelo
pipeline.fit(X_train, y_train)

# Comprobar los resultados con los datos de prueba
predictions = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

/home/aurora/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8738069989395546
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       117
           1       0.70      0.77      0.73       138
           2       0.79      0.82      0.81       142
           3       0.72      0.77      0.75       145
           4       0.87      0.78      0.82       157
           5       0.81      0.82      0.82       136
           6       0.83      0.84      0.84       137
           7       0.86      0.90      0.88       164
           8       0.90      0.91      0.91       155
           9       0.87      0.94      0.90       141
          10       0.94      0.93      0.93       136
          11       0.99      0.93      0.96       156
          12       0.80      0.79      0.80       144
          13       0.91      0.94      0.93       145
          14       0.97      0.91      0.94       152
          15       0.90      0.93      0.92       150
          16       0.92      0.95      0.94       15